In [1]:
import librosa
import os
import numpy as np
import sys
from dtw import dtw
from numpy.linalg import norm
from numpy import array
import pyaudio
import wave

import heapq

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



**|Heap| = 80**  
**n_mfcc = 20**  
**DTWSeq = mfcc**  
**Window-Norm**

In [2]:
def initialCorpus(path):
    # 音乐库位置
    audioList = os.listdir(path)

    raw_audioList = {}
    beat_database = {}

    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav'):
            # 读入一维音频序列
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save('beatDatabase_mfcc_20.npy', beat_database)
    
    return beat_database

In [3]:
def readCorpus(path):
    
    # 读入音乐节奏数据库
    all_data = np.load(path, allow_pickle=True)
    beat_database = all_data.item()
    
    return beat_database

In [4]:
def updateCorpus(path, dbpath):
    
    # 音乐库位置
    audioList = os.listdir(path)
    
    # 已保存序列的文件
    raw_db = readCorpus(dbPath)
    raw_files = raw_db.keys()
    
    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav') and audioName not in raw_files:
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save(dbpath, beat_database)

In [5]:
def normlize(data):
    n_mean = np.mean(data, axis=0)
    n_std  = np.std(data, axis=0)
    
    norm_data = np.divide(np.subtract(data, n_mean), n_std)
    return norm_data

In [6]:
def voiceCompare_quick(dbPath, tPath):
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # 读入要识别的录音
    y, sr = librosa.load(tPath)

    # 识别录音的节奏序列
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_frames = librosa.feature.delta(beat_frames,mode ='nearest')
    x = array(beat_frames).reshape(-1, 1)

    # 将待识别的录音序列与语料库中语音逐一做DTW对比
    compare_result = {}
    
    for songID in beat_database.keys():
        y = beat_database[songID]
        y = array(y).reshape(-1, 1)
        
        dist = dtw(x, y).distance
        # print('两段话的差异程度为： ', songID.split("\\")[1], ": ", dist)
        
        compare_result[songID] = dist

    matched_song = min(compare_result, key=compare_result.get)
    print("最接近的录音是：", matched_song)

In [8]:
from sklearn import preprocessing

def voiceCompare(dbPath, tPath):
    # 最大检索数
    aimNum = 80
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # ==== 读入要识别的录音 ====
    y, sr = librosa.load(tPath)

    # 提取录音的 MFCC 特征
    x = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)  # n1 * 20
    
    # 标准化
    for i in range(0, 20):
        # x[i] = preprocessing.minmax_scale(x[i])
        x[i] = normlize(x[i])
    
    x = x.T
    lenx = len(x)

    # ==== 将待识别的录音序列与语料库中语音逐一做DTW对比 ====
    
    # heap for [dist, 时间段，文件名]
    heap = []
    heapq.heapify(heap)  
    
    for songID in beat_database.keys():
        # 取出文件名对应的 mfcc 序列
        y = beat_database[songID]
        
        # 标准化
        for i in range(0, 20):
            # y[i] = preprocessing.minmax_scale(y[i])
            y[i] = normlize(y[i])
            
        y = y.T    
        leny = len(y) # n2 * 20 
        print(leny)

        for tp in range(0, leny - lenx):
            # *加速* 设定距离上限
            full = False  # 堆是否已满
            dist_UB = -10000  # DTW 距离上限
            overBound = False  # 是否过限
            
            if (len(heap) >= aimNum):
                full = True
                dist_UB = -heap[0][0]  # heap top (biggest) DTW dist as UB  
                
            # 计算 DTW(y[tp : tp + lenx])
            total_dist = 0
            
            for i in range(0, lenx):
                # DTW dist
                total_dist += dtw(x[i], y[tp + i], distance_only=False).distance
                
                # *加速* 超过上限直接取消
                if (full and total_dist > dist_UB):
                    overBound = True
                    break
            
            # *加速* 超过上限
            if (overBound):
                continue
            
            # 入栈
            tupleY = (-total_dist, tp, songID) # dtw 距离加负数转为大根堆
            
            heapq.heappush(heap, tupleY)
            if (len(heap) > aimNum):
                heapq.heappop(heap)
            
            print(tupleY)
            
        # end for
        
        
        
    return heap

In [9]:
def getTimePoint_dense(dbPath, tPath, vheap):
    res_num = 20 # 定义取出前 res_num 位的结果作为识别结果
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    # 提取前 res_num 个相似的片段并输出对应时间段
    similar_n = heapq.nlargest(res_num, vheap)
    
    print("开始输出相似片段：")
    
    for i in range(0, res_num):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数

        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")


In [15]:
def getTimePoint(dbPath, tPath, vheap):
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    heapq.nlargest(20, vheap)
    
    # ====== 对 vheap 进行去重 ======
    # 取出文件名
    name_set = set()
    for tp in vheap:
        name_set.add(tp[2])
    # print(name_set)
    
    # 合并下标差小于5的片段
    sheap = []
    for name in name_set:
        # 按下标排序
        nList = [x for x in vheap if x[2] == name]
        sortL = sorted(nList, key=lambda t:t[1])
        
        # 去重
        for tp in sortL:
            if len(sheap) < 1 or sheap[-1][2] != name or abs(sheap[-1][1] - tp[1]) > 5:
                sheap.append(tp)
            else:  
                if (sheap[-1][0] < tp[0]): 
                    sheap[-1] = tp  # 保留距离较小项

    # print(sheap)
    # 提取相似片段并输出对应时间段
    heapq.heapify(sheap)
    if (len(sheap) > 10):
        sheap = heapq.nlargest (10, sheap)
        
    similar_n = sheap    
    print("开始输出相似片段：")
    
    for i in range(0, len(sheap)):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数
        
        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        # print(music_name, music_time, music_pos, "/", music_all, "=", frag_st)
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")

In [11]:
# 语料库路径
corpus_path = './corpus'

# 数据表路径
dbPath = './beatDatabase_mfcc_20.npy';

# test file path
# testPath = './input/00415250-前5s.wav'
# testPath = './input/00429126-53s_60s.wav'
testPath = './input/00430105-hou5s.wav'

In [13]:
# 1 初始化语料序列库
# beatDB = initialCorpus(corpus_path)

# 2 更新语料库中新音乐文件的序列
# updateCorpus(corpus_path, dbPath)

# 3 读入语料序列库
# beat_database = readCorpus(dbPath)
testPath = './input/00430105-hou5s.wav'
vheap = voiceCompare(dbPath, testPath)

3876
(-1552.395266573545, 0, './corpus\\00415250.wav')
(-1573.9153801972461, 1, './corpus\\00415250.wav')
(-1590.4417497285358, 2, './corpus\\00415250.wav')
(-1606.343303674268, 3, './corpus\\00415250.wav')
(-1617.040728059732, 4, './corpus\\00415250.wav')
(-1623.5257282499406, 5, './corpus\\00415250.wav')
(-1641.580820257328, 6, './corpus\\00415250.wav')
(-1657.9537389136567, 7, './corpus\\00415250.wav')
(-1668.6474987713336, 8, './corpus\\00415250.wav')
(-1691.8583879553662, 9, './corpus\\00415250.wav')
(-1709.761838433118, 10, './corpus\\00415250.wav')
(-1740.8867901854865, 11, './corpus\\00415250.wav')
(-1758.3295897136159, 12, './corpus\\00415250.wav')
(-1779.7628959873873, 13, './corpus\\00415250.wav')
(-1805.2332557277005, 14, './corpus\\00415250.wav')
(-1828.3066111823027, 15, './corpus\\00415250.wav')
(-1841.5009139474896, 16, './corpus\\00415250.wav')
(-1848.2566539900326, 17, './corpus\\00415250.wav')
(-1859.9433391462262, 18, './corpus\\00415250.wav')
(-1867.2553972634669, 

(-1481.8394642039384, 658, './corpus\\00415250.wav')
(-1473.3872183617386, 659, './corpus\\00415250.wav')
(-1477.1145144032962, 660, './corpus\\00415250.wav')
(-1469.52610157269, 661, './corpus\\00415250.wav')
(-1456.5905465847063, 662, './corpus\\00415250.wav')
(-1451.065459388792, 663, './corpus\\00415250.wav')
(-1451.562275689812, 664, './corpus\\00415250.wav')
(-1453.8889216268617, 665, './corpus\\00415250.wav')
(-1459.9238681746829, 666, './corpus\\00415250.wav')
(-1478.9981415805123, 667, './corpus\\00415250.wav')
(-1501.6192561544194, 668, './corpus\\00415250.wav')
(-1505.378413483082, 669, './corpus\\00415250.wav')
(-1504.3197622237967, 695, './corpus\\00415250.wav')
(-1500.1697337339465, 697, './corpus\\00415250.wav')
(-1491.6621207412572, 698, './corpus\\00415250.wav')
(-1494.6413725950006, 704, './corpus\\00415250.wav')
(-1481.6684262984963, 705, './corpus\\00415250.wav')
(-1457.859010512755, 706, './corpus\\00415250.wav')
(-1448.5637816496783, 707, './corpus\\00415250.wav')

(-1395.8822731899745, 1711, './corpus\\00415250.wav')
(-1395.651766736777, 1712, './corpus\\00415250.wav')
(-1395.9839706018283, 1713, './corpus\\00415250.wav')
(-1396.0630696981862, 1714, './corpus\\00415250.wav')
(-1396.0173698484614, 1733, './corpus\\00415250.wav')
(-1395.8341447978928, 1734, './corpus\\00415250.wav')
(-1396.1536367654517, 1735, './corpus\\00415250.wav')
(-1396.215665568383, 1736, './corpus\\00415250.wav')
(-1396.215665568383, 1737, './corpus\\00415250.wav')
(-1396.215665568383, 1738, './corpus\\00415250.wav')
(-1396.215665568383, 1739, './corpus\\00415250.wav')
(-1396.215665568383, 1740, './corpus\\00415250.wav')
(-1396.215665568383, 1741, './corpus\\00415250.wav')
(-1396.215665568383, 1742, './corpus\\00415250.wav')
(-1396.215665568383, 1743, './corpus\\00415250.wav')
(-1396.215665568383, 1744, './corpus\\00415250.wav')
(-1396.215665568383, 1745, './corpus\\00415250.wav')
(-1396.215665568383, 1746, './corpus\\00415250.wav')
(-1396.215665568383, 1747, './corpus\\00

10164
7150
(-1371.1264477989062, 911, './corpus\\00430105.wav')


In [14]:
getTimePoint(dbPath, testPath, vheap)

开始输出相似片段：
./corpus\00415250.wav , 26.35 秒, 28.42 秒
./corpus\00415250.wav , 76.56 秒, 78.62 秒
./corpus\00430105.wav , 21.15 秒, 23.22 秒
./corpus\00429239.wav , 12.24 秒, 14.31 秒
./corpus\00429239.wav , 102.35 秒, 104.42 秒
./corpus\00429239.wav , 207.21 秒, 209.28 秒
./corpus\00429239.wav , 207.35 秒, 209.42 秒
./corpus\00429239.wav , 226.48 秒, 228.55 秒
./corpus\00429239.wav , 226.62 秒, 228.69 秒
./corpus\00429126.wav , 80.27 秒, 82.34 秒
./corpus\00429126.wav , 94.73 秒, 96.80 秒


In [ ]:
inPara = sys.argv

if (len(inPara) < 2):
    print("请输入待识别录音文件路径！")
else:
    if (len(inPara) > 2):
        print("给定语料库路径为:", sys.argv[2])
        corpus_path = sys.argv[2]
    else:
        print("默认语料库路径为：", corpus_path)
    
    if (len(inPara) > 3):
        print("给定数据表路径为:", sys.argv[3])
        dbPath = sys.argv[3]
    else:
        print("默认数据表路径为：", dbPath)
    
    testPath = sys.argv[1]
    vheap = voiceCompare(dbPath, testPath)
    getTimePoint(dbPath, testPath, vheap)
  

In [12]:
testPath2 = './input/00430105-hou5s.wav'
vheap2 = voiceCompare(dbPath, testPath2)
getTimePoint(dbPath, testPath2, vheap2)

3876
(-1552.395266573545, 0, './corpus\\00415250.wav')
(-1573.9153801972461, 1, './corpus\\00415250.wav')
(-1590.4417497285358, 2, './corpus\\00415250.wav')
(-1606.343303674268, 3, './corpus\\00415250.wav')
(-1617.040728059732, 4, './corpus\\00415250.wav')
(-1623.5257282499406, 5, './corpus\\00415250.wav')
(-1641.580820257328, 6, './corpus\\00415250.wav')
(-1657.9537389136567, 7, './corpus\\00415250.wav')
(-1668.6474987713336, 8, './corpus\\00415250.wav')
(-1691.8583879553662, 9, './corpus\\00415250.wav')
(-1709.761838433118, 10, './corpus\\00415250.wav')
(-1740.8867901854865, 11, './corpus\\00415250.wav')
(-1758.3295897136159, 12, './corpus\\00415250.wav')
(-1779.7628959873873, 13, './corpus\\00415250.wav')
(-1805.2332557277005, 14, './corpus\\00415250.wav')
(-1828.3066111823027, 15, './corpus\\00415250.wav')
(-1841.5009139474896, 16, './corpus\\00415250.wav')
(-1848.2566539900326, 17, './corpus\\00415250.wav')
(-1859.9433391462262, 18, './corpus\\00415250.wav')
(-1867.2553972634669, 

(-1519.174622292292, 287, './corpus\\00415250.wav')
(-1508.9218337316306, 288, './corpus\\00415250.wav')
(-1483.4515375992523, 289, './corpus\\00415250.wav')
(-1472.1224068579722, 290, './corpus\\00415250.wav')
(-1469.3969176195787, 291, './corpus\\00415250.wav')
(-1478.947329980827, 292, './corpus\\00415250.wav')
(-1486.2517027624817, 293, './corpus\\00415250.wav')
(-1491.3365158215565, 294, './corpus\\00415250.wav')
(-1477.455028778215, 295, './corpus\\00415250.wav')
(-1470.3020552632333, 296, './corpus\\00415250.wav')
(-1466.7190168128654, 297, './corpus\\00415250.wav')
(-1466.7035737837941, 298, './corpus\\00415250.wav')
(-1467.4991146379107, 299, './corpus\\00415250.wav')
(-1465.714547865613, 300, './corpus\\00415250.wav')
(-1459.6453071483593, 301, './corpus\\00415250.wav')
(-1450.6630515591423, 302, './corpus\\00415250.wav')
(-1440.0475310394722, 303, './corpus\\00415250.wav')
(-1434.4177095149826, 304, './corpus\\00415250.wav')
(-1432.373889566883, 305, './corpus\\00415250.wav'

(-1498.2526183244372, 921, './corpus\\00415250.wav')
(-1502.5582529825806, 922, './corpus\\00415250.wav')
(-1492.8164399828129, 923, './corpus\\00415250.wav')
(-1485.9583351965302, 924, './corpus\\00415250.wav')
(-1465.1074145530183, 925, './corpus\\00415250.wav')
(-1415.5732193411277, 926, './corpus\\00415250.wav')
(-1419.3882448490076, 927, './corpus\\00415250.wav')
(-1435.608224214684, 928, './corpus\\00415250.wav')
(-1415.0035897441448, 929, './corpus\\00415250.wav')
(-1428.7770506492416, 930, './corpus\\00415250.wav')
(-1432.5051773491614, 931, './corpus\\00415250.wav')
(-1465.0033431922966, 932, './corpus\\00415250.wav')
(-1489.2159333509226, 933, './corpus\\00415250.wav')
(-1494.8452910927663, 978, './corpus\\00415250.wav')
(-1482.5076814436288, 979, './corpus\\00415250.wav')
(-1467.138529387903, 980, './corpus\\00415250.wav')
(-1456.8233455243098, 981, './corpus\\00415250.wav')
(-1454.3664392932515, 982, './corpus\\00415250.wav')
(-1452.4987456723065, 983, './corpus\\00415250.w

(-1396.9412765994384, 1720, './corpus\\00415250.wav')
(-1397.164665241511, 1721, './corpus\\00415250.wav')
(-1397.0491210490181, 1722, './corpus\\00415250.wav')
(-1396.589527738066, 1723, './corpus\\00415250.wav')
(-1396.8775260865405, 1724, './corpus\\00415250.wav')
(-1396.356718805404, 1725, './corpus\\00415250.wav')
(-1396.593884487189, 1726, './corpus\\00415250.wav')
(-1396.4012253254368, 1727, './corpus\\00415250.wav')
(-1396.6299807247888, 1728, './corpus\\00415250.wav')
(-1396.729831126304, 1729, './corpus\\00415250.wav')
(-1396.7160171538308, 1730, './corpus\\00415250.wav')
(-1396.6088635444357, 1731, './corpus\\00415250.wav')
(-1396.4662108167722, 1732, './corpus\\00415250.wav')
(-1396.0173698484614, 1733, './corpus\\00415250.wav')
(-1395.8341447978928, 1734, './corpus\\00415250.wav')
(-1396.1536367654517, 1735, './corpus\\00415250.wav')
(-1396.215665568383, 1736, './corpus\\00415250.wav')
(-1396.215665568383, 1737, './corpus\\00415250.wav')
(-1396.215665568383, 1738, './corpu

(-1369.1683608532294, 4411, './corpus\\00429239.wav')
(-1388.2247910994447, 4412, './corpus\\00429239.wav')
(-1393.6174368676839, 4413, './corpus\\00429239.wav')
(-1390.567762642262, 7739, './corpus\\00429239.wav')
(-1380.4048170937858, 8319, './corpus\\00429239.wav')
(-1379.9245645290757, 8320, './corpus\\00429239.wav')
(-1394.0750410976652, 8321, './corpus\\00429239.wav')
(-1386.7993939715197, 8922, './corpus\\00429239.wav')
(-1348.1994862362735, 8923, './corpus\\00429239.wav')
(-1333.7499119131353, 8924, './corpus\\00429239.wav')
(-1364.8311917966166, 8925, './corpus\\00429239.wav')
(-1369.5705551700776, 8926, './corpus\\00429239.wav')
(-1372.0538779307499, 8927, './corpus\\00429239.wav')
(-1380.8710307697875, 8928, './corpus\\00429239.wav')
(-1389.697291898552, 8929, './corpus\\00429239.wav')
(-1368.2131287044504, 8930, './corpus\\00429239.wav')
(-1375.7351412683602, 8931, './corpus\\00429239.wav')
(-1393.653016390057, 8932, './corpus\\00429239.wav')
(-1385.541842888324, 9750, './c

In [14]:
getTimePoint(dbPath, testPath2, vheap2)

开始输出相似片段：
./corpus\00415250.wav , 24.73 秒, 26.80 秒
./corpus\00415250.wav , 87.03 秒, 89.10 秒
./corpus\00429239.wav , 193.19 秒, 195.26 秒
./corpus\00430105.wav , 163.28 秒, 165.35 秒
./corpus\00429881.wav , 171.85 秒, 173.91 秒
./corpus\00415250.wav , 26.35 秒, 28.42 秒
./corpus\00415250.wav , 30.39 秒, 32.46 秒
./corpus\00415250.wav , 71.75 秒, 73.82 秒
./corpus\00429239.wav , 226.62 秒, 228.69 秒
./corpus\00429881.wav , 86.21 秒, 88.28 秒
